## **라이브러리 불러오기**

In [ ]:
# 데이터 핸들링
import numpy as np
import pandas as pd
import datetime
import re
from time import sleep
import time
import random
import csv

print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

pandas version:  2.2.2


In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **데이터 결합**

In [ ]:
# 데이터 불러오기
df02 = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/review_dataset02.csv')
df03 = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/review_dataset03.csv')
df04 = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/review_dataset04.csv')
df05 = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/review_dataset05.csv')

# 데이터 전처리
del df02["Unnamed: 0"]; del df03["Unnamed: 0"]; del df04["Unnamed: 0"]; del df05["Unnamed: 0"]                                                            # 데이터 제거
del df02["리뷰_ID"]; del df03["리뷰_ID"]; del df04["리뷰_ID"]; del df05["리뷰_ID"]

df02["작성일자(전체)"] = pd.to_datetime(df02["작성일자(전체)"])                                                                                           # 데이터를 타입 변환
df03["작성일자(전체)"] = pd.to_datetime(df03["작성일자(전체)"])
df04["작성일자(전체)"] = pd.to_datetime(df04["작성일자(전체)"])
df05["작성일자(전체)"] = pd.to_datetime(df05["작성일자(전체)"])

final_review_df = pd.concat([df02, df03, df04, df05])                                                                                                     # 데이터 결합
final_review_df = final_review_df.sort_values("작성일자(전체)")                                                                                           # 작성일 순으로 정렬
final_review_df = final_review_df.drop_duplicates()                                                                                                       # 중복 데이터 제거
final_review_df = final_review_df.reset_index()                                                                                                           # 인덱스 초기화
del final_review_df["index"]                                                                                                                              # 데이터 제거
final_review_df["리뷰_ID"] = list(range(len(final_review_df)))                                                                                            # (중복된 데이터 제거 完) 리뷰별 고유번호 변수 생성
final_review_df = final_review_df[['리뷰_ID', '고객_ID', '평점', '리뷰어', '유용성', '작성일자(전체)', '작성일자(년)', '작성일자(월)', '작성일자(일)']]   # 데이터 변수별 순서 변경
final_review_df["Binary"] = np.where(final_review_df["작성일자(전체)"]>="2022-10-16", "After", "Befor")                                                   # KGA대상 전/후 구분 변수 생성

# 데이터 확인
print(final_review_df.shape)
final_review_df.tail()

(11667, 10)


,리뷰_ID,고객_ID,평점,리뷰어,유용성,작성일자(전체),작성일자(년),작성일자(월),작성일자(일),Binary
11662,11662,ᄌᄒ,1,1주년이 뭔 주는게 저번 레압뿌리는거만도못함 그리고 오즈마 횟수는 왜갑자기 증발하는...,1,2023-04-04,2023,4,4,After
11663,11663,YU k,3,사람이 너무 많이 줄었음;; '그 국가'에 정식출시하는 것 말곤 답이 없는 듯,1,2023-04-04,2023,4,4,After
11664,11664,장정현,1,생각이 있다면 버서커 너프좀 하시길,1,2023-04-04,2023,4,4,After
11665,11665,Eddie park,3,오래전 피던에대한 추억 향수으로 잘하고 있지만 이벤트도 너무 숙제같아지고 있고 신규...,1,2023-04-05,2023,4,5,After
11666,11666,롤롤,5,아라드격투왕 터치안되는버그 빨리 고쳐주시고 접속하면 아이템 없다고 뜨는 버그 고쳐줘요,0,2023-04-05,2023,4,5,After


In [ ]:
## 데이터 확인
# KGA대상 전/후 구분 상태 확인
final_review_df["Binary"].value_counts().sort_index(ascending=True)

,count
Binary,
After,876
Befor,10791


In [ ]:
## 데이터 확인
# 작성일자(전체) 확인
final_review_df.describe()

,리뷰_ID,평점,작성일자(전체),작성일자(년),작성일자(월),작성일자(일)
count,11667.00000,11667.000000,11667,11667.000000,11667.000000,11667.000000
mean,5833.00000,3.676095,2022-04-28 18:04:02.530213632,2022.045770,3.700694,21.479129
min,0.00000,1.000000,2022-03-22 00:00:00,2022.000000,1.000000,1.000000
25%,2916.50000,2.000000,2022-03-24 00:00:00,2022.000000,3.000000,23.000000
50%,5833.00000,5.000000,2022-03-26 00:00:00,2022.000000,3.000000,24.000000
75%,8749.50000,5.000000,2022-04-10 00:00:00,2022.000000,4.000000,25.000000
max,11666.00000,5.000000,2023-04-05 00:00:00,2023.000000,12.000000,31.000000
std,3368.11713,1.666651,NaN,0.208995,1.799785,7.919976


In [ ]:
# 데이터 백업
#final_review_df.to_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/Fianl_Review_Dataset_01',  encoding = 'utf-8-sig')

In [ ]:
## 코드 리뷰

# 설명: 본 페이지는 크롤링을 통해 수집한 게임 리뷰 데이터를 확인하고, 구조에 맞게 데이터를 결합하는 코드
# 특징: 크롤링한 데이터를 보면 동일 조건으로 진행했지만, 수집된 데이터 수가 상이하다는 것을 확인 完
#       따라서, 수집된 데이터를 모두 결합하여, 최종 데이터를 확보하고자 가공 및 구축 完

___